In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import json
import requests
import time

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Matt\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Matt\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Matt\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Matt\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\Matt\AppData\Roaming\nltk_data...
[nltk_data]    |   Pac

True

In [4]:
review_df = pd.read_csv("data/steam_review_data1.csv.gz")
game_df = pd.read_csv("data/steamspy_data.csv")

In [5]:
game_df.sort_values(by="ccu", inplace=True, ascending=False)
game_df.head(10)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
25,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,NaN,5472091,733433,0,"50,000,000 .. 100,000,000",28266,710,6106,260,0.0,0.0,0.0,"English, Czech, Danish, Dutch, Finnish, French...","Action, Free to Play",821347,"{'FPS': 87303, 'Shooter': 62660, 'Multiplayer'..."
22,570,Dota 2,Valve,Valve,NaN,1398684,273412,0,"100,000,000 .. 200,000,000",37809,1523,932,798,0.0,0.0,0.0,"English, Bulgarian, Czech, Danish, Dutch, Finn...","Action, Free to Play, Strategy",615508,"{'Free to Play': 58240, 'MOBA': 19355, 'Multip..."
53656,1599340,Lost Ark,Smilegate RPG,Amazon Games,NaN,1866,77,0,"0 .. 20,000",160,131,214,151,0.0,0.0,0.0,"English, French, German, Spanish - Spain","Action, Adventure, Free to Play, Massively Mul...",517487,"{'MMORPG': 96, 'Action': 92, 'RPG': 87, 'Actio..."
13158,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",NaN,1086669,868546,0,"50,000,000 .. 100,000,000",23290,532,7785,184,0.0,0.0,0.0,"English, Korean, Simplified Chinese, French, G...","Action, Adventure, Free to Play, Massively Mul...",382831,"{'Survival': 13774, 'Shooter': 11598, 'Multipl..."
36345,1172470,Apex Legends,Respawn Entertainment,Electronic Arts,NaN,334106,51549,0,"20,000,000 .. 50,000,000",4597,575,637,279,0.0,0.0,0.0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Free to Play",195941,"{'Free to Play': 1079, 'Battle Royale': 762, '..."
11532,534380,Dying Light 2 Stay Human,Techland,Techland,NaN,32998,7450,0,"1,000,000 .. 2,000,000",735,735,595,595,5999.0,5999.0,0.0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, RPG",165529,"{'Open World': 570, 'Zombies': 552, 'Co-op': 5..."
2709,271590,Grand Theft Auto V,Rockstar North,Rockstar Games,NaN,1089871,203526,0,"20,000,000 .. 50,000,000",12850,658,5851,212,1480.0,1480.0,0.0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure",115688,"{'Open World': 29754, 'Action': 21753, 'Multip..."
2238,252490,Rust,Facepunch Studios,Facepunch Studios,NaN,651289,96732,0,"20,000,000 .. 50,000,000",17524,1559,3499,933,3999.0,3999.0,0.0,"English, French, Italian, German, Spanish - Sp...","Action, Adventure, Indie, Massively Multiplaye...",107834,"{'Survival': 16028, 'Crafting': 10126, 'Multip..."
1964,238960,Path of Exile,Grinding Gear Games,Grinding Gear Games,NaN,161542,16091,0,"10,000,000 .. 20,000,000",8899,1091,332,878,0.0,0.0,0.0,"English, Portuguese - Brazil, Russian, Thai, F...","Action, Adventure, Free to Play, Indie, Massiv...",92565,"{'Free to Play': 16752, 'Action RPG': 3214, 'H..."
37595,1203220,NARAKA: BLADEPOINT,24 Entertainment,NetEase Games Montréal,NaN,70709,18208,0,"10,000,000 .. 20,000,000",5917,911,2358,757,1999.0,1999.0,0.0,"English, Simplified Chinese, Traditional Chine...","Action, Adventure, Massively Multiplayer",90703,"{'Battle Royale': 476, 'Multiplayer': 413, 'Se..."


In [6]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1336828 entries, 0 to 1336827
Data columns (total 18 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   Unnamed: 0                      1336828 non-null  int64  
 1   appid                           1336828 non-null  int64  
 2   recommendationid                1336828 non-null  int64  
 3   author                          1336828 non-null  int64  
 4   author_playtime_forever         1336828 non-null  int64  
 5   author_playtime_last_two_weeks  1336828 non-null  int64  
 6   author_last_played              1336828 non-null  int64  
 7   author_num_reviews              1336828 non-null  int64  
 8   votes_up                        1336828 non-null  int64  
 9   votes_funny                     1336828 non-null  int64  
 10  weighted_vote_score             1336828 non-null  float64
 11  comment_count                   1336828 non-null  int64  
 12  

In [7]:
game_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63968 entries, 25 to 63967
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   appid            63968 non-null  int64  
 1   name             63865 non-null  object 
 2   developer        56704 non-null  object 
 3   publisher        56711 non-null  object 
 4   score_rank       48 non-null     float64
 5   positive         63968 non-null  int64  
 6   negative         63968 non-null  int64  
 7   userscore        63968 non-null  int64  
 8   owners           63968 non-null  object 
 9   average_forever  63968 non-null  int64  
 10  average_2weeks   63968 non-null  int64  
 11  median_forever   63968 non-null  int64  
 12  median_2weeks    63968 non-null  int64  
 13  price            56854 non-null  float64
 14  initialprice     56856 non-null  float64
 15  discount         56856 non-null  float64
 16  languages        56814 non-null  object 
 17  genre           

In [8]:
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [
        token for token in tokens if token not in stopwords.words("english")
    ]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [
        lemmatizer.lemmatize(token) for token in filtered_tokens
    ]

    # Join the tokens back into a string
    processed_text = " ".join(lemmatized_tokens)
    return processed_text

In [9]:
# initialize NLTK sentiment analyzer
analyzer = SentimentIntensityAnalyzer()


# create get_sentiment function
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)

    return scores

In [10]:
review_df = review_df.dropna()

In [11]:
review_df["review_processed"] = review_df["review"].apply(preprocess_text)
review_df["sentiment"] = review_df["review_processed"].apply(get_sentiment)

In [12]:
reduced_df = review_df[["appid", "sentiment"]]
reduced_df.to_csv(
    "data/steam_review_sentiment.csv.gz", index=False, compression="gzip"
)